In [2]:
import quandl
from datetime import date
from bokeh.charts import Bar
from bokeh.io import output_notebook, show, vform
output_notebook()
from pandas import ExcelWriter

Loading BokehJS ...

__plan and notes__

- get physical volume from USDA convert to contracts

- calc total volume traded last year divided by phy volume

- calc average OI for the year divide by phys volume

- if possible, calc total deliveries for the year divide by phys volume

do 1. 2. 3. for producer/merchant/processor

Notes:

- [oats specs](www.cmegroup.com/trading/agricultural/grain-and-oilseed/oats_contract_specifications.html)
- cot dataset titles like "Commitment of Traders - Oats - Futures Only"
 - dept of ag NASS dataset titles like "Oats - Production, Measured In Bu"

- [CME contract specs](http://www.cmegroup.com/trading/agricultural/grain-and-oilseed/oats_contract_specifications.html) example

- Some of the 2015 crop data is missing, I'm filling in 2015 with infromation form this pdf, i dont know why 2015 isnt online yet, but other years are the same in this report as the dataset. ["Crop Production 2015 Summary"](http://www.usda.gov/nass/PUBS/TODAYRPT/cropan16.pdf) [didnt end up doing this but could be done in future revision]
    
- 3,929,885 - 2015 Soybeans


__Set up and Pull data__

In [3]:
#SET UP FOR QUANDL PULL

#volume is from wiki continous futures
# .7 is the volume column
Volume_codes = ["CHRIS/CME_O1.7", #oats 1
            "CHRIS/CME_O2.7", #oats 2
            "CHRIS/CME_O3.7", #oats 3 
                
            "CHRIS/CME_S1.7", #soybean 1
            "CHRIS/CME_S2.7", #soybean 2
            "CHRIS/CME_S3.7", #soybean 3
            "CHRIS/CME_S4.7", #soybean 4
            "CHRIS/CME_S5.7", #soybean 5
            "CHRIS/CME_S6.7", #soybean 6
            "CHRIS/CME_S7.7", #soybean 7
            "CHRIS/CME_S8.7", #soybean 8
            "CHRIS/CME_S9.7", #soybean 9
            "CHRIS/CME_S10.7", #soybean 10
            
            "CHRIS/CME_C1.7", #corn 1
            "CHRIS/CME_C2.7", #corn 2
            "CHRIS/CME_C3.7", #corn 3
            "CHRIS/CME_C4.7", #corn 4
            "CHRIS/CME_C5.7", #corn 5
            "CHRIS/CME_C6.7", #corn 6
            "CHRIS/CME_C7.7", #corn 7
            "CHRIS/CME_C8.7", #corn 8
            "CHRIS/CME_C9.7", #corn 9
            "CHRIS/CME_C10.7", #corn 10
                
            "CHRIS/CME_W1.7", #Wheat (all KC) 1
            "CHRIS/CME_W2.7", #Wheat 2
            "CHRIS/CME_W3.7", #Wheat 3
            "CHRIS/CME_W4.7", #Wheat 4
            "CHRIS/CME_W5.7", #Wheat 5
            "CHRIS/CME_W6.7", #Wheat 6
            "CHRIS/CME_W7.7", #Wheat 7
            "CHRIS/CME_W8.7", #Wheat 8

            "CHRIS/CME_RR1.7", #rice 1
            "CHRIS/CME_RR2.7", #rice 2
            "CHRIS/CME_RR3.7", #rice 3       
           ]

#oi from COT
OI_codes = ["CFTC/O_F_ALL", #oats
            "CFTC/S_F_ALL", #soybean
            "CFTC/C_F_ALL", #corn
            "CFTC/W_F_ALL", #wheat
            "CFTC/RR_F_ALL", #rice
           ]

#production from USDA NASS (National Ag Stats Service)
Prod_codes = ["USDANASS/NASS_OATSPRODUCTIONMEASUREDINBU", #oats
              "USDANASS/NASS_SOYBEANSPRODUCTIONMEASUREDINBU", #soybean
              "USDANASS/NASS_CORNGRAINPRODUCTIONMEASUREDINBU", #corn
              "USDANASS/NASS_WHEATWINTERREDSOFTPRODUCTIONMEASUREDINBU", #wheat
              "USDANASS/NASS_RICEPRODUCTIONMEASUREDINCWT", #rice
             ]

#four years ago for quandl pull
fouryears = date(2012, 1, 1)

In [4]:
OI = quandl.get(OI_codes, trim_start=fouryears, authtoken="UDaCz3fqxxJUAL3-WWmD", returns="pandas")

In [5]:
Production = quandl.get(Prod_codes, trim_start=fouryears, authtoken="UDaCz3fqxxJUAL3-WWmD", returns="pandas")

In [6]:
Volume = quandl.get(Volume_codes,trim_start=fouryears, authtoken="UDaCz3fqxxJUAL3-WWmD", returns="pandas")

__Production Data__

In [10]:
Production.head()

,USDANASS/NASS_OATSPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_SOYBEANSPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_CORNGRAINPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_WHEATWINTERREDSOFTPRODUCTIONMEASUREDINBU - Value,USDANASS/NASS_RICEPRODUCTIONMEASUREDINCWT - Value
Date,,,,,
2012-12-31,61486000.0,3.042044e+09,1.075511e+10,412729000.0,199939000.0
2013-12-31,64642000.0,3.357984e+09,1.382896e+10,568481000.0,189953000.0
2014-12-31,69684000.0,3.968823e+09,1.421553e+10,455297000.0,221035000.0
2015-12-31,83640000.0,NaN,NaN,393416000.0,NaN


In [7]:
#PRODUCTION
#copy because quandl has a call limit and sometimes and I want this dataset to be my base.
prod = Production.copy()

#index based on date
prod.index.name = None

#already had this code thought it'd be easier than cut and paste them ugly col names
oat_cols = [col for col in prod.columns if 'OATS' in col]
soybean_cols = [col for col in prod.columns if 'SOYBEAN' in col]
corn_cols = [col for col in prod.columns if 'CORN' in col]
wheat_cols = [col for col in prod.columns if 'WHEAT' in col]
rice_cols = [col for col in prod.columns if 'RICE' in col]

#convert phy volum produced into contracts
#contract sizes from CME website
prod['Contracts Oats'] = prod[oat_cols]/5000
prod['Contracts Soybean'] = prod[soybean_cols]/5000
prod['Contracts Corn'] = prod[corn_cols]/5000
prod['Contracts Wheat'] = prod[wheat_cols]/5000
prod['Contracts Rice'] = prod[rice_cols]/2000

In [ ]:
prod = prod.drop(oat_cols,1)
prod = prod.drop(soybean_cols,1)
prod = prod.drop(corn_cols,1)
prod = prod.drop(wheat_cols,1)
prod = prod.drop(rice_cols,1)

__OI Data__

In [ ]:
#OI
oi = OI.copy()

#index based on date
oi.index.name = None

#resample to yearly data, take mean of colums
oi = oi.resample('A').mean()